In [ ]:
"""定义句子窗口检索"""
class SentenceWindow:
    def metadata_format(self, ordered_text, **kwargs):
        count = kwargs.get("split_count", 1)
        for i, document in enumerate(ordered_text):
            if i - count >= 0:
                document.metadata['previous_page'] = ordered_text[i-count].page_content
            else:
                document.metadata['previous_page'] = ''
             
            if i + count < len(ordered_text):
                document.metadata['next_page'] = ordered_text[i+count].page_content
            else:
                document.metadata['next_page'] = ''

            if not document.metadata.get('techniques'):
                document.metadata['techniques'] = ''
        return ordered_text
    def search_and_format(self, retriever, query, **kwargs):
        top_documents = retriever.get_relevant_documents(query)
        formatted_docs = []
        for doc in top_documents:
            # 获取前后文
            previous_page = doc.metadata.get("previous_page", "")
            next_page = doc.metadata.get("next_page", "")
            techniques = doc.metadata.get("techniques", "")
            # 拼接前后文与当前文档
            doc.page_content = previous_page + doc.page_content + next_page + techniques
            formatted_docs.append(doc)
        return formatted_docs

In [ ]:
"""多路召回检索"""
class MultiRoad:
    def search_and_format(self, databases, query, **kwargs):
        top_documents = retriever.get_relevant_documents(query)
        result_data = []
        pairs = [[query, item.page_content] for item in top_documents]
        with torch.no_grad():
            rerank_tokenizer = AutoTokenizer.from_pretrained('bce-reranker-base_v1')
            inputs = rerank_tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
            rerank_model = AutoModelForSequenceClassification.from_pretrained('bce-reranker-base_v1')
            scores = rerank_model(**inputs, return_dict=True).logits.view(-1, ).float()
            for i, score in enumerate(scores):
                data = {
                "text": top_documents[i].page_content,
                "score": float(score)
                }
                result_data.append(data)
        returnresult_data